In [4]:
import numpy as np
import mujoco
import mediapy
import matplotlib.pyplot as plt
import time

In [85]:
def load_model_and_reset_data_from_xml(xml_string):
    mj_model = mujoco.MjModel.from_xml_string(xml_string)
    mj_data = mujoco.MjData(mj_model)

    mujoco.mj_resetData(mj_model, mj_data)
    return mj_data, mj_model
    # mj_data.joint('root').qvel = initial_qvel
def render(mj_data, mj_model, framerate=30, play_speed=1, num_frames=30 * 10):
    mj_renderer = mujoco.Renderer(mj_model)

    frames = []
    for idx in range(num_frames):
        while mj_data.time * framerate / play_speed < idx:
            mujoco.mj_step(mj_model, mj_data)

        # mj_renderer.update_scene(mj_data, camera="fixed")
        mj_renderer.update_scene(mj_data, camera=0)
        frames.append(mj_renderer.render())

    mediapy.show_video(frames, fps=framerate)
    mj_renderer.close()

# Joint
https://mujoco.readthedocs.io/en/stable/XMLreference.html#body-joint

## Joint order matters

Note: hinge axis is 0 -1 0, CCW is +

In [132]:
mjcf_xml = """
<mujoco>
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>
  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 0" xyaxes="1 0 0 0 0 1"/>

    <!-- Add coordinate axes visualization -->
    <body name="coordinate_frame" pos="-0.4 0 -0.3">
        <geom name="x_axis" type="capsule" fromto="0 0 0 0.2 0 0" size="0.004" rgba="1 0 0 1"/>  <!-- Red for X axis -->
        <geom name="y_axis" type="capsule" fromto="0 0 0 0 0.2 0" size="0.004" rgba="0 1 0 1"/>  <!-- Green for Y axis -->
        <geom name="z_axis" type="capsule" fromto="0 0 0 0 0 0.2" size="0.004" rgba="0 0 1 1"/>  <!-- Blue for Z axis -->
    </body>

    <body name="0" pos="0 0 .2">
      <joint type="slide" axis="1 0 0" range="-0.1 0.1" name="slide_joint"/>
      <joint name="root"/>
      <geom fromto="0 0 0 0 0 -.25" rgba="1 1 0 1"/>


      <body name="3" pos="0 0 -.25">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 0 1 1"/>
      </body>
    </body>
  </worldbody>
</mujoco>
"""
mj_data, mj_model = load_model_and_reset_data_from_xml(mjcf_xml)
mj_data.joint('root').qvel = 10
render(mj_data, mj_model)


This browser does not support the video tag.


In [78]:
mjcf_xml = """
<mujoco>
  <compiler angle="radian"/>
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>
  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 0" xyaxes="1 0 0 0 0 1"/>
    <body name="0" pos="0 0 .2">
      <joint name="root"/>
      <joint type="slide" axis="1 0 0" range="-0.1 0.1" name="slide_joint"/>
      <geom fromto="0 0 0 0 0 -.25" rgba="1 1 0 1"/>


      <body name="3" pos="0 0 -.25">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 0 1 1"/>
      </body>
    </body>
  </worldbody>
</mujoco>
"""
mj_data, mj_model = load_model_and_reset_data_from_xml(mjcf_xml)
mj_data.joint('root').qvel = 10
render(mj_data, mj_model)


This browser does not support the video tag.


In [79]:
mjcf_xml = """
<mujoco>
  <compiler angle="radian"/>
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>
  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 0" xyaxes="1 0 0 0 0 1"/>
    <body name="0" pos="0 0 .2">
      <joint name="root"/>
      <joint type="slide" axis="0 0 1" range="-0.1 0.1" name="slide_joint"/>
      <geom fromto="0 0 0 0 0 -.25" rgba="1 1 0 1"/>


      <body name="3" pos="0 0 -.25">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 0 1 1"/>
      </body>
    </body>
  </worldbody>
</mujoco>
"""
mj_data, mj_model = load_model_and_reset_data_from_xml(mjcf_xml)
mj_data.joint('root').qvel = 10
render(mj_data, mj_model)


This browser does not support the video tag.


## Joint stiffness, damping, ref

ref: initial value of joint(different with changing qpos, ref set initial position as ref)

In [137]:
mjcf_xml = """
<mujoco>
    <compiler angle="radian"/>
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>

  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 0" xyaxes="1 0 0 0 0 1"/>
    <body name="0" pos="0 0 .2">
      <joint name="root" stiffness="2" damping="0.9" ref="6.28" springref="0"/>
      <geom fromto="0 0 0 0 0 -.25" rgba="1 1 0 1"/>


      <body name="3" pos="0 0 -.25">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 0 1 1"/>
      </body>
    </body>
  </worldbody>
</mujoco>
"""
mj_data, mj_model = load_model_and_reset_data_from_xml(mjcf_xml)
mj_data.joint('root').qvel = 0
render(mj_data, mj_model, num_frames=30 * 4)


This browser does not support the video tag.


In [138]:
mjcf_xml = """
<mujoco>
    <compiler angle="radian"/>
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>

  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 0" xyaxes="1 0 0 0 0 1"/>
    <body name="0" pos="0 0 .2">
      <joint name="root" stiffness="2" damping="0.9" springref="6.28"/>
      <geom fromto="0 0 0 0 0 -.25" rgba="1 1 0 1"/>


      <body name="3" pos="0 0 -.25">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 0 1 1"/>
      </body>
    </body>
  </worldbody>
</mujoco>
"""
mj_data, mj_model = load_model_and_reset_data_from_xml(mjcf_xml)
mj_data.joint('root').qvel = 0
render(mj_data, mj_model, num_frames=30 * 4)

This browser does not support the video tag.


In [139]:
mjcf_xml = """
<mujoco>
    <compiler angle="radian"/>
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>

  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 0" xyaxes="1 0 0 0 0 1"/>
    <body name="0" pos="0 0 .2">
      <joint name="root" stiffness="2" damping="0.9" springref="6.28" ref="6.28"/>
      <geom fromto="0 0 0 0 0 -.25" rgba="1 1 0 1"/>


      <body name="3" pos="0 0 -.25">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 0 1 1"/>
      </body>
    </body>
  </worldbody>
</mujoco>
"""
mj_data, mj_model = load_model_and_reset_data_from_xml(mjcf_xml)
mj_data.joint('root').qvel = 0
render(mj_data, mj_model, num_frames=30 * 4)

## joint contact

vanila

In [142]:
mjcf_xml = """
<mujoco>
    <compiler angle="radian"/>
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>

  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 -0.2" xyaxes="1 0 0 0 0 1"/>

    <!-- Add coordinate axes visualization -->
    <body name="coordinate_frame" pos="-0.4 0 -0.03">
        <geom name="x_axis" type="capsule" fromto="0 0 0 0.2 0 0" size="0.004" rgba="1 0 0 1"/>  <!-- Red for X axis -->
        <geom name="y_axis" type="capsule" fromto="0 0 0 0 0.2 0" size="0.004" rgba="0 1 0 1"/>  <!-- Green for Y axis -->
        <geom name="z_axis" type="capsule" fromto="0 0 0 0 0 0.2" size="0.004" rgba="0 0 1 1"/>  <!-- Blue for Z axis -->
    </body>


    <body name="0" pos="0 0 0">
      <joint type="slide" axis="0 0 1" range="-0.03 0.03" name="slide_joint"/>
      <joint name="root" stiffness="2" damping="0.9" ref="0"/>
      <geom fromto="0 0 0 0 0 -.25" rgba="1 1 0 1"/>


      <body name="3" pos="0 0 -.25">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 0 1 1"/>
        <body name="weight" pos="0 0 -.2">
          <geom type="sphere" size=".05" rgba="1 0 0 1" mass="50"/>
        </body>
      </body>
    </body>
  </worldbody>
</mujoco>
"""
mj_data, mj_model = load_model_and_reset_data_from_xml(mjcf_xml)
mj_data.joint('root').qvel = 0
render(mj_data, mj_model, play_speed=1, num_frames=30 * 3)


## solreflimit

In [143]:
mjcf_xml = """
<mujoco>
    <compiler angle="radian"/>
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>

  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 -0.2" xyaxes="1 0 0 0 0 1"/>

    <!-- Add coordinate axes visualization -->
    <body name="coordinate_frame" pos="-0.4 0 -0.03">
        <geom name="x_axis" type="capsule" fromto="0 0 0 0.2 0 0" size="0.004" rgba="1 0 0 1"/>  <!-- Red for X axis -->
        <geom name="y_axis" type="capsule" fromto="0 0 0 0 0.2 0" size="0.004" rgba="0 1 0 1"/>  <!-- Green for Y axis -->
        <geom name="z_axis" type="capsule" fromto="0 0 0 0 0 0.2" size="0.004" rgba="0 0 1 1"/>  <!-- Blue for Z axis -->
    </body>


    <body name="0" pos="0 0 0">
      <joint type="slide" axis="0 0 1" range="-0.03 0.03" name="slide_joint" solreflimit="0.2"/>
      <joint name="root" stiffness="2" damping="0.9" ref="0"/>
      <geom fromto="0 0 0 0 0 -.25" rgba="1 1 0 1"/>


      <body name="3" pos="0 0 -.25">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 0 1 1"/>
        <body name="weight" pos="0 0 -.2">
          <geom type="sphere" size=".05" rgba="1 0 0 1" mass="50"/>
        </body>
      </body>
    </body>
  </worldbody>
</mujoco>
"""
mj_data, mj_model = load_model_and_reset_data_from_xml(mjcf_xml)
mj_data.joint('root').qvel = 0
render(mj_data, mj_model, play_speed=1, num_frames=30 * 3)


In [146]:
mjcf_xml = """
<mujoco>
    <compiler angle="radian"/>
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>

  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 -0.2" xyaxes="1 0 0 0 0 1"/>

    <!-- Add coordinate axes visualization -->
    <body name="coordinate_frame" pos="-0.4 0 -0.03">
        <geom name="x_axis" type="capsule" fromto="0 0 0 0.2 0 0" size="0.004" rgba="1 0 0 1"/>  <!-- Red for X axis -->
        <geom name="y_axis" type="capsule" fromto="0 0 0 0 0.2 0" size="0.004" rgba="0 1 0 1"/>  <!-- Green for Y axis -->
        <geom name="z_axis" type="capsule" fromto="0 0 0 0 0 0.2" size="0.004" rgba="0 0 1 1"/>  <!-- Blue for Z axis -->
    </body>


    <body name="0" pos="0 0 0">
      <joint type="slide" axis="0 0 1" range="-0.03 0.03" name="slide_joint" solreflimit="0.5"/>
      <joint name="root" stiffness="2" damping="0.9" ref="0"/>
      <geom fromto="0 0 0 0 0 -.25" rgba="1 1 0 1"/>


      <body name="3" pos="0 0 -.25">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 0 1 1"/>
        <body name="weight" pos="0 0 -.2">
          <geom type="sphere" size=".05" rgba="1 0 0 1" mass="50"/>
        </body>
      </body>
    </body>
  </worldbody>
</mujoco>
"""
mj_data, mj_model = load_model_and_reset_data_from_xml(mjcf_xml)
mj_data.joint('root').qvel = 0
render(mj_data, mj_model, play_speed=1, num_frames=30 * 3)


solimplimit, solreffriction, solimpfriction -> no big difference (need graph?)

## margin

In [157]:
slide_range = 0.1
mjcf_xml = f"""
<mujoco>
    <compiler angle="radian"/>
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>

  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 -{slide_range}" xyaxes="1 0 0 0 0 1"/>

    <!-- Add coordinate axes visualization -->
    <body name="coordinate_frame" pos="-0.4 0 -{slide_range}">
        <geom name="x_axis" type="capsule" fromto="0 0 0 0.2 0 0" size="0.004" rgba="1 0 0 1"/>  <!-- Red for X axis -->
        <geom name="y_axis" type="capsule" fromto="0 0 0 0 0.2 0" size="0.004" rgba="0 1 0 1"/>  <!-- Green for Y axis -->
        <geom name="z_axis" type="capsule" fromto="0 0 0 0 0 0.2" size="0.004" rgba="0 0 1 1"/>  <!-- Blue for Z axis -->
    </body>


    <body name="0" pos="0 0 0">
      <joint type="slide" axis="0 0 1" range="-{slide_range} {slide_range}" name="slide_joint" margin="0.0"/>
      <joint name="root" stiffness="2" damping="0.9" ref="0"/>
      <geom fromto="0 0 0 0 0 -.25" rgba="1 1 0 1"/>


      <body name="3" pos="0 0 -.25">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 0 1 1"/>
        <body name="weight" pos="0 0 -.2">
          <geom type="sphere" size=".05" rgba="1 0 0 1" mass="50"/>
        </body>
      </body>
    </body>
  </worldbody>
</mujoco>
"""
mj_data, mj_model = load_model_and_reset_data_from_xml(mjcf_xml)
mj_data.joint('root').qvel = 0
render(mj_data, mj_model, play_speed=0.1, num_frames=30 * 3)


In [158]:
slide_range = 0.1
mjcf_xml = f"""
<mujoco>
    <compiler angle="radian"/>
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>

  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 -{slide_range}" xyaxes="1 0 0 0 0 1"/>

    <!-- Add coordinate axes visualization -->
    <body name="coordinate_frame" pos="-0.4 0 -{slide_range}">
        <geom name="x_axis" type="capsule" fromto="0 0 0 0.2 0 0" size="0.004" rgba="1 0 0 1"/>  <!-- Red for X axis -->
        <geom name="y_axis" type="capsule" fromto="0 0 0 0 0.2 0" size="0.004" rgba="0 1 0 1"/>  <!-- Green for Y axis -->
        <geom name="z_axis" type="capsule" fromto="0 0 0 0 0 0.2" size="0.004" rgba="0 0 1 1"/>  <!-- Blue for Z axis -->
    </body>


    <body name="0" pos="0 0 0">
      <joint type="slide" axis="0 0 1" range="-{slide_range} {slide_range}" name="slide_joint" margin="0.05"/>
      <joint name="root" stiffness="2" damping="0.9" ref="0"/>
      <geom fromto="0 0 0 0 0 -.25" rgba="1 1 0 1"/>


      <body name="3" pos="0 0 -.25">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 0 1 1"/>
        <body name="weight" pos="0 0 -.2">
          <geom type="sphere" size=".05" rgba="1 0 0 1" mass="50"/>
        </body>
      </body>
    </body>
  </worldbody>
</mujoco>
"""
mj_data, mj_model = load_model_and_reset_data_from_xml(mjcf_xml)
mj_data.joint('root').qvel = 0
render(mj_data, mj_model, play_speed=0.1, num_frames=30 * 3)


In [159]:
slide_range = 0.1
mjcf_xml = f"""
<mujoco>
    <compiler angle="radian"/>
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>

  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 -{slide_range}" xyaxes="1 0 0 0 0 1"/>

    <!-- Add coordinate axes visualization -->
    <body name="coordinate_frame" pos="-0.4 0 -{slide_range}">
        <geom name="x_axis" type="capsule" fromto="0 0 0 0.2 0 0" size="0.004" rgba="1 0 0 1"/>  <!-- Red for X axis -->
        <geom name="y_axis" type="capsule" fromto="0 0 0 0 0.2 0" size="0.004" rgba="0 1 0 1"/>  <!-- Green for Y axis -->
        <geom name="z_axis" type="capsule" fromto="0 0 0 0 0 0.2" size="0.004" rgba="0 0 1 1"/>  <!-- Blue for Z axis -->
    </body>


    <body name="0" pos="0 0 0">
      <joint type="slide" axis="0 0 1" range="-{slide_range} {slide_range}" name="slide_joint" margin="0.2"/>
      <joint name="root" stiffness="2" damping="0.9" ref="0"/>
      <geom fromto="0 0 0 0 0 -.25" rgba="1 1 0 1"/>


      <body name="3" pos="0 0 -.25">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 0 1 1"/>
        <body name="weight" pos="0 0 -.2">
          <geom type="sphere" size=".05" rgba="1 0 0 1" mass="50"/>
        </body>
      </body>
    </body>
  </worldbody>
</mujoco>
"""
mj_data, mj_model = load_model_and_reset_data_from_xml(mjcf_xml)
mj_data.joint('root').qvel = 0
render(mj_data, mj_model, play_speed=0.1, num_frames=30 * 3)


## Armature

In [161]:
mjcf_xml = f"""
<mujoco>
    <compiler angle="radian"/>
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>

  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 0" xyaxes="1 0 0 0 0 1"/>

    <!-- Add coordinate axes visualization -->
    <body name="coordinate_frame" pos="-0.4 0 0">
        <geom name="x_axis" type="capsule" fromto="0 0 0 0.2 0 0" size="0.004" rgba="1 0 0 1"/>  <!-- Red for X axis -->
        <geom name="y_axis" type="capsule" fromto="0 0 0 0 0.2 0" size="0.004" rgba="0 1 0 1"/>  <!-- Green for Y axis -->
        <geom name="z_axis" type="capsule" fromto="0 0 0 0 0 0.2" size="0.004" rgba="0 0 1 1"/>  <!-- Blue for Z axis -->
    </body>


    <body name="0" pos="0 0 0">
      <joint name="root" stiffness="2" damping="0.9" ref="0" armature="0.0"/>
      <geom fromto="0 0 0 0 0 -.25" rgba="1 1 0 1"/>


      <body name="3" pos="0 0 -.25">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 0 1 1"/>
        <body name="weight" pos="0 0 -.2">
          <geom type="sphere" size=".05" rgba="1 0 0 1" mass="50"/>
        </body>
      </body>
    </body>
  </worldbody>
</mujoco>
"""
mj_data, mj_model = load_model_and_reset_data_from_xml(mjcf_xml)
mj_data.joint('root').qvel = 10
render(mj_data, mj_model, play_speed=1, num_frames=30 * 3)


This browser does not support the video tag.


In [164]:
mjcf_xml = f"""
<mujoco>
    <compiler angle="radian"/>
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>

  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 0" xyaxes="1 0 0 0 0 1"/>

    <!-- Add coordinate axes visualization -->
    <body name="coordinate_frame" pos="-0.4 0 0">
        <geom name="x_axis" type="capsule" fromto="0 0 0 0.2 0 0" size="0.004" rgba="1 0 0 1"/>  <!-- Red for X axis -->
        <geom name="y_axis" type="capsule" fromto="0 0 0 0 0.2 0" size="0.004" rgba="0 1 0 1"/>  <!-- Green for Y axis -->
        <geom name="z_axis" type="capsule" fromto="0 0 0 0 0 0.2" size="0.004" rgba="0 0 1 1"/>  <!-- Blue for Z axis -->
    </body>


    <body name="0" pos="0 0 0">
      <joint name="root" stiffness="2" damping="0.9" ref="0" armature="10"/>
      <geom fromto="0 0 0 0 0 -.25" rgba="1 1 0 1"/>


      <body name="3" pos="0 0 -.25">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 0 1 1"/>
        <body name="weight" pos="0 0 -.2">
          <geom type="sphere" size=".05" rgba="1 0 0 1" mass="50"/>
        </body>
      </body>
    </body>
  </worldbody>
</mujoco>
"""
mj_data, mj_model = load_model_and_reset_data_from_xml(mjcf_xml)
mj_data.joint('root').qvel = 10
render(mj_data, mj_model, play_speed=1, num_frames=30 * 3)


This browser does not support the video tag.


## Frictionloss

In [168]:
mjcf_xml = f"""
<mujoco>
    <compiler angle="radian"/>
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>

  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 0" xyaxes="1 0 0 0 0 1"/>

    <!-- Add coordinate axes visualization -->
    <body name="coordinate_frame" pos="-0.4 0 0">
        <geom name="x_axis" type="capsule" fromto="0 0 0 0.2 0 0" size="0.004" rgba="1 0 0 1"/>  <!-- Red for X axis -->
        <geom name="y_axis" type="capsule" fromto="0 0 0 0 0.2 0" size="0.004" rgba="0 1 0 1"/>  <!-- Green for Y axis -->
        <geom name="z_axis" type="capsule" fromto="0 0 0 0 0 0.2" size="0.004" rgba="0 0 1 1"/>  <!-- Blue for Z axis -->
    </body>


    <body name="0" pos="0 0 0">
      <joint name="root" stiffness="2" damping="0.9" frictionloss="100.0" ref="0" armature="10"/>
      <geom fromto="0 0 0 0 0 -.25" rgba="1 1 0 1"/>


      <body name="3" pos="0 0 -.25">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 0 1 1"/>
        <body name="weight" pos="0 0 -.2">
          <geom type="sphere" size=".05" rgba="1 0 0 1" mass="50"/>
        </body>
      </body>
    </body>
  </worldbody>
</mujoco>
"""
mj_data, mj_model = load_model_and_reset_data_from_xml(mjcf_xml)
mj_data.joint('root').qvel = 10
render(mj_data, mj_model, play_speed=1, num_frames=30 * 3)


This browser does not support the video tag.


### versus damping

In [170]:
mjcf_xml = f"""
<mujoco>
    <compiler angle="radian"/>
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>

  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 -0.2" xyaxes="1 0 0 0 0 1"/>

    <!-- Add coordinate axes visualization -->
    <body name="coordinate_frame" pos="-0.4 0 0">
        <geom name="x_axis" type="capsule" fromto="0 0 0 0.2 0 0" size="0.004" rgba="1 0 0 1"/>  <!-- Red for X axis -->
        <geom name="y_axis" type="capsule" fromto="0 0 0 0 0.2 0" size="0.004" rgba="0 1 0 1"/>  <!-- Green for Y axis -->
        <geom name="z_axis" type="capsule" fromto="0 0 0 0 0 0.2" size="0.004" rgba="0 0 1 1"/>  <!-- Blue for Z axis -->
    </body>


    <body name="0" pos="0 0 0">
      <joint name="root" stiffness="2" damping="100" frictionloss="0" ref="0" armature="10"/>
      <geom fromto="0 0 0 0 0 -.25" rgba="1 1 0 1"/>


      <body name="3" pos="0 0 -.25">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 0 1 1"/>
        <body name="weight" pos="0 0 -.2">
          <geom type="sphere" size=".05" rgba="1 0 0 1" mass="50"/>
        </body>
      </body>
    </body>
  </worldbody>
</mujoco>
"""
mj_data, mj_model = load_model_and_reset_data_from_xml(mjcf_xml)
mj_data.joint('root').qvel = 10
render(mj_data, mj_model, play_speed=1, num_frames=30 * 3)


This browser does not support the video tag.


# Contact

free joint

In [204]:
mjcf_xml = f"""
<mujoco>
    <compiler angle="radian"/>
  <option timestep=".001">
    <flag energy="enable" contact="enable"/>

  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 -0.2" xyaxes="1 0 0 0 0 1"/>

    <!-- Add coordinate axes visualization -->
    <body name="coordinate_frame" pos="-0.4 0 0">
        <geom name="x_axis" type="capsule" fromto="0 0 0 0.2 0 0" size="0.004" rgba="1 0 0 1"/>  <!-- Red for X axis -->
        <geom name="y_axis" type="capsule" fromto="0 0 0 0 0.2 0" size="0.004" rgba="0 1 0 1"/>  <!-- Green for Y axis -->
        <geom name="z_axis" type="capsule" fromto="0 0 0 0 0 0.2" size="0.004" rgba="0 0 1 1"/>  <!-- Blue for Z axis -->
    </body>

    <!-- Add ground plane -->
    <geom name="ground" type="plane" pos="0 0 -0.3" size="100 100 0.1" rgba=".9 .9 .9 1"/>

    <body name="weight" pos="0 0 0">
        <joint name="free_joint" type="free"/>
        <!-- <joint name="vertical" type="slide" axis="0 0 1"/>
        <joint name="horizontal" type="slide" axis="1 0 0"/> -->
        <geom type="sphere" size=".05" rgba="1 0 0 1" mass="50"/>
    </body>
  </worldbody>
</mujoco>
"""
mj_data, mj_model = load_model_and_reset_data_from_xml(mjcf_xml)
mj_data.joint('free_joint').qpos = np.array([-0.3,0,0, 0,0,0,0])
mj_data.joint('free_joint').qvel = np.array([10,0,-10, 0,0,0])
# mj_data.joint('horizontal').qpos = -0.3
# mj_data.joint('vertical').qvel = -10
# mj_data.joint('horizontal').qvel = 10

render(mj_data, mj_model, play_speed=0.1, num_frames=30 * 3)

2D with no rotation

In [203]:
mjcf_xml = f"""
<mujoco>
    <compiler angle="radian"/>
  <option timestep=".001">
    <flag energy="enable" contact="enable"/>

  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 -0.2" xyaxes="1 0 0 0 0 1"/>

    <!-- Add coordinate axes visualization -->
    <body name="coordinate_frame" pos="-0.4 0 0">
        <geom name="x_axis" type="capsule" fromto="0 0 0 0.2 0 0" size="0.004" rgba="1 0 0 1"/>  <!-- Red for X axis -->
        <geom name="y_axis" type="capsule" fromto="0 0 0 0 0.2 0" size="0.004" rgba="0 1 0 1"/>  <!-- Green for Y axis -->
        <geom name="z_axis" type="capsule" fromto="0 0 0 0 0 0.2" size="0.004" rgba="0 0 1 1"/>  <!-- Blue for Z axis -->
    </body>

    <!-- Add ground plane -->
    <geom name="ground" type="plane" pos="0 0 -0.3" size="100 100 0.1" rgba=".9 .9 .9 1"/>

    <body name="weight" pos="0 0 0">
        <!-- <joint name="free_joint" type="free"/> -->
        <joint name="vertical" type="slide" axis="0 0 1"/>
        <joint name="horizontal" type="slide" axis="1 0 0"/>
        <geom type="sphere" size=".05" rgba="1 0 0 1" mass="50"/>
    </body>
  </worldbody>
</mujoco>
"""
mj_data, mj_model = load_model_and_reset_data_from_xml(mjcf_xml)

# mj_data.joint('free_joint').qpos = np.array([-0.3,0,0, 0,0,0,0])
# mj_data.joint('free_joint').qvel = np.array([10,0,-10, 0,0,0])
mj_data.joint('horizontal').qpos = -0.3
mj_data.joint('vertical').qvel = -10
mj_data.joint('horizontal').qvel = 10

render(mj_data, mj_model, play_speed=0.1, num_frames=30 * 3)

### solif

solref : real(2), “0.02 1”

(timeconst,dampratio)

In [ ]:
mjcf_xml = f"""
<mujoco>
    <compiler angle="radian"/>
  <option timestep=".001">
    <flag energy="enable" contact="enable"/>

  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 -0.2" xyaxes="1 0 0 0 0 1"/>

    <!-- Add coordinate axes visualization -->
    <body name="coordinate_frame" pos="-0.4 0 0">
        <geom name="x_axis" type="capsule" fromto="0 0 0 0.2 0 0" size="0.004" rgba="1 0 0 1"/>  <!-- Red for X axis -->
        <geom name="y_axis" type="capsule" fromto="0 0 0 0 0.2 0" size="0.004" rgba="0 1 0 1"/>  <!-- Green for Y axis -->
        <geom name="z_axis" type="capsule" fromto="0 0 0 0 0 0.2" size="0.004" rgba="0 0 1 1"/>  <!-- Blue for Z axis -->
    </body>

    <!-- Add ground plane -->
    <geom name="ground" type="plane" pos="0 0 -0.3" size="100 100 0.1" rgba=".9 .9 .9 1"/>

    <body name="weight" pos="0 0 0">
        <!-- <joint name="free_joint" type="free"/> -->
        <joint name="vertical" type="slide" axis="0 0 1"/>
        <joint name="horizontal" type="slide" axis="1 0 0"/>
        <!-- solref : real(2), “0.02 1” solimp : real(5), “0.9 0.95 0.001 0.5 2” -->
        <geom type="sphere" size=".05" rgba="1 0 0 1" mass="1" solref="0.0000001 1"/>
    </body>
  </worldbody>
</mujoco>
"""
mj_data, mj_model = load_model_and_reset_data_from_xml(mjcf_xml)

# mj_data.joint('free_joint').qpos = np.array([-0.3,0,0, 0,0,0,0])
# mj_data.joint('free_joint').qvel = np.array([10,0,-10, 0,0,0])
mj_data.joint('horizontal').qpos = -0.3
mj_data.joint('vertical').qvel = -10
mj_data.joint('horizontal').qvel = 10

render(mj_data, mj_model, play_speed=0.1, num_frames=30 * 3)

In [219]:
mjcf_xml = f"""
<mujoco>
    <compiler angle="radian"/>
  <option timestep=".001">
    <flag energy="enable" contact="enable"/>

  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 -0.2" xyaxes="1 0 0 0 0 1"/>

    <!-- Add coordinate axes visualization -->
    <body name="coordinate_frame" pos="-0.4 0 0">
        <geom name="x_axis" type="capsule" fromto="0 0 0 0.2 0 0" size="0.004" rgba="1 0 0 1"/>  <!-- Red for X axis -->
        <geom name="y_axis" type="capsule" fromto="0 0 0 0 0.2 0" size="0.004" rgba="0 1 0 1"/>  <!-- Green for Y axis -->
        <geom name="z_axis" type="capsule" fromto="0 0 0 0 0 0.2" size="0.004" rgba="0 0 1 1"/>  <!-- Blue for Z axis -->
    </body>

    <!-- Add ground plane -->
    <geom name="ground" type="plane" pos="0 0 -0.3" size="100 100 0.1" rgba=".9 .9 .9 1"/>

    <body name="weight" pos="0 0 0">
        <!-- <joint name="free_joint" type="free"/> -->
        <joint name="vertical" type="slide" axis="0 0 1"/>
        <joint name="horizontal" type="slide" axis="1 0 0"/>
        <!-- solref : real(2), “0.02 1” solimp : real(5), “0.9 0.95 0.001 0.5 2” -->
        <geom type="sphere" size=".05" rgba="1 0 0 1" mass="1" solref="0.02 3"/>
    </body>
  </worldbody>
</mujoco>
"""
mj_data, mj_model = load_model_and_reset_data_from_xml(mjcf_xml)

# mj_data.joint('free_joint').qpos = np.array([-0.3,0,0, 0,0,0,0])
# mj_data.joint('free_joint').qvel = np.array([10,0,-10, 0,0,0])
mj_data.joint('horizontal').qpos = -0.3
mj_data.joint('vertical').qvel = -10
mj_data.joint('horizontal').qvel = 10

render(mj_data, mj_model, play_speed=0.1, num_frames=30 * 3)

In [220]:
mjcf_xml = f"""
<mujoco>
    <compiler angle="radian"/>
  <option timestep=".001">
    <flag energy="enable" contact="enable"/>

  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 -0.2" xyaxes="1 0 0 0 0 1"/>

    <!-- Add coordinate axes visualization -->
    <body name="coordinate_frame" pos="-0.4 0 0">
        <geom name="x_axis" type="capsule" fromto="0 0 0 0.2 0 0" size="0.004" rgba="1 0 0 1"/>  <!-- Red for X axis -->
        <geom name="y_axis" type="capsule" fromto="0 0 0 0 0.2 0" size="0.004" rgba="0 1 0 1"/>  <!-- Green for Y axis -->
        <geom name="z_axis" type="capsule" fromto="0 0 0 0 0 0.2" size="0.004" rgba="0 0 1 1"/>  <!-- Blue for Z axis -->
    </body>

    <!-- Add ground plane -->
    <geom name="ground" type="plane" pos="0 0 -0.3" size="100 100 0.1" rgba=".9 .9 .9 1"/>

    <body name="weight" pos="0 0 0">
        <!-- <joint name="free_joint" type="free"/> -->
        <joint name="vertical" type="slide" axis="0 0 1"/>
        <joint name="horizontal" type="slide" axis="1 0 0"/>
        <!-- solref : real(2), “0.02 1” solimp : real(5), “0.9 0.95 0.001 0.5 2” -->
        <geom type="sphere" size=".05" rgba="1 0 0 1" mass="1" solref="0.02 0.0001"/>
    </body>
  </worldbody>
</mujoco>
"""
mj_data, mj_model = load_model_and_reset_data_from_xml(mjcf_xml)

# mj_data.joint('free_joint').qpos = np.array([-0.3,0,0, 0,0,0,0])
# mj_data.joint('free_joint').qvel = np.array([10,0,-10, 0,0,0])
mj_data.joint('horizontal').qpos = -0.3
mj_data.joint('vertical').qvel = -10
mj_data.joint('horizontal').qvel = 10

render(mj_data, mj_model, play_speed=0.1, num_frames=30 * 3)